In [ ]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import numpy as np
import torch

In [ ]:
model = YOLO('yolov8n-seg.pt')

In [ ]:
model = YOLO('runs/segment/train/weights/best.pt') #if you already train a model, run this cell

In [ ]:
Normal_dir=Path("TB_Chest_Radiography_Database/Normal/")
TB_dir=Path("TB_Chest_Radiography_Database/Tuberculosis/")
Normal_images:list=sorted(list(map(str, list(Normal_dir.glob("*.png")))))
TB_images:list=sorted(list(map(str, list(TB_dir.glob("*.png")))))

In [ ]:
for path in Normal_images:
    path=path.replace("\\",'/')
    img=cv2.imread(path)
    img=cv2.GaussianBlur(img,(5,5),0)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2LAB)
    clahe=cv2.createCLAHE(3,(16,16))
    img[:,:,0] = clahe.apply(img[:,:,0])
    img=cv2.cvtColor(img,cv2.COLOR_LAB2RGB)
    img=cv2.resize(img,(224,224))
    cv2.imwrite(f'Preprocessed/Normal/{path.split("/")[-1]}',img)

In [ ]:
for path in TB_images:
    path=path.replace("\\",'/')
    img=cv2.imread(path)
    img=cv2.GaussianBlur(img,(5,5),0)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2LAB)
    clahe=cv2.createCLAHE(3,(16,16))
    img[:,:,0] = clahe.apply(img[:,:,0])
    img=cv2.cvtColor(img,cv2.COLOR_LAB2RGB)
    img=cv2.resize(img,(224,224))
    cv2.imwrite(f'Preprocessed/Tuberculosis/{path.split("/")[-1]}',img)

In [ ]:
model.train(data='Chest X-Ray Segmentation.v5i.yolov8\data.yaml',
            epochs=100,
            patience=10,
            batch=10,
            imgsz=224,
            device=0,
            optimizer='Adam',
            lr0=1e-3,
            lrf=1e-3,
            amp=False)

In [ ]:
model.val()

In [ ]:
preprocessed_normal_dir=Path("Preprocessed/Normal")
preprocessed_Tuberculosis_dir=Path("Preprocessed/Tuberculosis")
preprocessed_normal_images:list=sorted(list(map(str, list(preprocessed_normal_dir.glob("*.png")))))
preprocessed_Tuberculosis_images:list=sorted(list(map(str, list(preprocessed_Tuberculosis_dir.glob("*.png")))))
len(preprocessed_normal_images),len(preprocessed_Tuberculosis_images)

In [ ]:
results=model.predict(preprocessed_Tuberculosis_images,imgsz=224,device=0)
not_segmented=0
for i in range(len(results)):
    masks_raw=results[i].masks.data.cpu().numpy()
    if len(masks_raw)==2:
        mask_raw1=masks_raw[0].astype(np.uint8)
        mask_raw2=masks_raw[1].astype(np.uint8)
        mask_raw=np.add(mask_raw1,mask_raw2)
        mask_3channel = cv2.merge((mask_raw,mask_raw,mask_raw))
        h2, w2, c2 = results[i].orig_img.shape
        mask = cv2.resize(mask_3channel, (w2, h2))
        masked=np.multiply(results[i].orig_img,mask)
        cv2.imwrite(f'Cropped/Tuberculosis/Tuberculosis_{i+1}.png',masked)
    else:
        not_segmented+=1

In [ ]:
not_segmented

In [ ]:
results1=model.predict(preprocessed_normal_images,imgsz=224,device=0)
not_segmented1=0
for i in range(len(results1)):
    if results1[i] == None:
        not_segmented1+=1
    masks_raw=results1[i].masks.data.cpu().numpy()
    if len(masks_raw)==2:
        mask_raw1=masks_raw[0].astype(np.uint8)
        mask_raw2=masks_raw[1].astype(np.uint8)
        mask_raw=np.add(mask_raw1,mask_raw2)
        mask_3channel = cv2.merge((mask_raw,mask_raw,mask_raw))
        h2, w2, c2 = results1[i].orig_img.shape
        mask = cv2.resize(mask_3channel, (w2, h2))
        masked=np.multiply(results1[i].orig_img,mask)
        cv2.imwrite(f'Cropped/Normal/Normal_{i+1}.png',masked)
    else:
        not_segmented1+=1

In [ ]:
not_segmented1

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [ ]:
Cropped_normal_dir=Path("Cropped/Normal")
Cropped_normal_images:list=sorted(list(map(str, list(Cropped_normal_dir.glob("*.png")))))
Cropped_Tuberculosis_dir=Path("Cropped/Tuberculosis")
Cropped_Tuberculosis_images:list=sorted(list(map(str, list(Cropped_Tuberculosis_dir.glob("*.png")))))
len(Cropped_normal_images),len(Cropped_Tuberculosis_images)

In [ ]:
normal_class=[[1,0]]*len(Cropped_normal_images)
tb_class=[[0,1]]*len(Cropped_Tuberculosis_images)
len(normal_class),len(tb_class)

In [ ]:
images=Cropped_normal_images+Cropped_Tuberculosis_images
labels=normal_class+tb_class
len(images),len(labels)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(np.array(images),np.array(labels),test_size=0.1,random_state=42)
x_train.shape,x_val.shape,y_train.shape,y_val.shape

In [ ]:
def image_preprocessing(path):
    img=cv2.imread(path)
    img=np.moveaxis(img,2,0)
    return img

In [ ]:
x_train=np.array(list(map(image_preprocessing,x_train)))
x_val=np.array(list(map(image_preprocessing,x_val)))
x_train.shape,x_val.shape

In [ ]:
device=("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
x_train=torch.from_numpy(x_train.astype(np.float16)).to(device)
x_val=torch.from_numpy(x_val.astype(np.float32)).to(device)
type(x_train),type(x_val)

In [ ]:
y_train=torch.from_numpy(y_train.astype(np.float16)).to(device)
y_val=torch.from_numpy(y_val.astype(np.float32)).to(device)
type(y_train),type(y_val)

In [ ]:
class StandardScaler():
    def __init__(self) -> None:
        self.mean=None
        self.std=None
    def fit(self,tensor:torch.Tensor) -> None:
        self.mean=tensor.mean((0,2,3),keepdim=True)
        self.std=tensor.std((0,2,3),keepdim=True)
    def transform(self,tensor:torch.Tensor) -> torch.Tensor:
        scaled=(tensor-self.mean)/(self.std+1e-5)
        return scaled
    def fit_transform(self,tensor:torch.Tensor) -> torch.Tensor:
        self.fit(tensor=tensor)
        scaled=self.transform(tensor=tensor)
        return scaled

In [ ]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_val=scaler.transform(x_val)

In [ ]:
train_dataset=TensorDataset(x_train,y_train)
train_loader=DataLoader(train_dataset,batch_size=20)
val_dataset=TensorDataset(x_val,y_val)
val_loader=DataLoader(val_dataset,batch_size=20)

In [ ]:
from super_gradients.training.utils.distributed_training_utils import setup_device
setup_device(device='cuda',num_gpus=1)

In [ ]:
from super_gradients import Trainer

experiment_name = "resnet50_cxr"
CHECKPOINT_DIR = 'checkpoints/root/dir'

trainer = Trainer(experiment_name=experiment_name, ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
from super_gradients.training import models
from super_gradients.common.object_names import Models

model = models.get(model_name=Models.RESNET50, num_classes=2, pretrained_weights='imagenet')

In [ ]:
from super_gradients.training import training_hyperparams

training_params =  training_hyperparams.get(config_name="training_hyperparams/imagenet_resnet50_train_params")

In [ ]:
import pprint
pprint.pprint("Training parameters:")
pprint.pprint(training_params)

In [ ]:
training_params["max_epochs"] = 5
training_params["train_metrics_list"]=['Accuracy']
training_params["valid_metrics_list"]=['Accuracy']

In [ ]:
trainer.train(model=model,
              training_params=training_params, 
              train_loader=train_loader,
              valid_loader=val_loader)

In [ ]:
prediction_model = models.get(model_name=Models.RESNET50,
                       num_classes=2,
                       checkpoint_path='E:/Coolyeah/KULIAH/Komputasi Intelegensia/UTS/Risman/checkpoints/root/dir/resnet50_cxr/RUN_20240227_150705_010683/ckpt_best.pth')

In [ ]:
from super_gradients.training.metrics.classification_metrics import Accuracy
valid_result=trainer.test(prediction_model,val_loader,test_metrics_list=[Accuracy()])
print(valid_result)